# T11 Machine learning model

## 1. Problem definition
        
        We have to predict what percentage of users will return to the game tomorrow. This KPI is officially known as Retention Day 1. I think we should use regression model.
    
## 2. Data
        
        We've gotten time series data from 02.06.2022. to 15.09.2022. But for data from 31.08.2022 t0 15.09.2022. we have to predict tagret variable (returned).
## 3. Evaluation
        
        Our result have to be evaluated by RMSE (Root Mean Squared Error on Prediction) for the period after 2022-08-31. So we'll definitely use regression model. We're going to make validation set to check our model
        
## 4. Features
        registration_type:
                            1 = Fresh Registration 
                            2 = Inactive ReRegistration
                            3 = Active ReRegistration
       
       played_t11_before:
                           0 = data not provided
                           1 = user says he played T11 before
                           2 = user says he didn't play T11 before
     
     registration_channel: Label that marks if the new user was targeted by any T11 marketing campaigns, before installing the game.
                           0 = data not provided
                           1 = Organic
                           2 = Paid             CHECK
                           
      network_type:
                    0 = data not provided
                    1 = WIFI
                    2 = LTE (4G)
                    3 = HSPAP (3G)               CHECK
                    
      device_tier: Our internal label for the quality of the device used for playing the game
                    0 = data not provided
                    1 = LowestEnd
                    2 = LowEnd
                    3 = MidEnd
                    4 = HighEnd
      device_type:
                     1 = Phone
                     2 = Tablet
                     
      device_manufacturer:
                        0 = Other + data not provided
                        1 = Samsung
                        2 = Xiaomi
                        3 = OPPO
                        4 = Huawei
                        5 = Motorola
                        6 = Vivo
                        7 = Realme
                        8 = INFINIX MOBILITY LIMITED
                        9 = LGE
                        10 = TECNO MOBILE LIMITED
                        11 = Lenovo
                        12 = OnePlus
                        13 = TCL

        screen_dpi
        device_memory_size_mb
        device_model
        os_version
        registrations


## 5. Model
        SO we're going to use regression model, but I am not sure which one. 
        
## 6. Experiments
        
        We're trying couple model and chose one with best RMSE.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn
%matplotlib inline

In [2]:
df = pd.read_csv("2. job_fair_retention_prediction_2022_training.csv")

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 342332 entries, 0 to 342331
Data columns (total 14 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   date                   342332 non-null  object 
 1   registration_type      342332 non-null  int64  
 2   played_t11_before      342332 non-null  int64  
 3   registration_channel   342332 non-null  int64  
 4   network_type           342332 non-null  int64  
 5   device_tier            342332 non-null  int64  
 6   device_type            342332 non-null  int64  
 7   device_manufacturer    342332 non-null  int64  
 8   screen_dpi             340442 non-null  float64
 9   device_memory_size_mb  340442 non-null  float64
 10  device_model           340442 non-null  object 
 11  os_version             340442 non-null  object 
 12  registrations          342332 non-null  int64  
 13  returned               342332 non-null  int64  
dtypes: float64(2), int64(9), object(3)
m

In [4]:
df.isnull().sum()

date                        0
registration_type           0
played_t11_before           0
registration_channel        0
network_type                0
device_tier                 0
device_type                 0
device_manufacturer         0
screen_dpi               1890
device_memory_size_mb    1890
device_model             1890
os_version               1890
registrations               0
returned                    0
dtype: int64

In [24]:
df.describe()

,registration_type,played_t11_before,registration_channel,network_type,device_tier,device_type,device_manufacturer,screen_dpi,device_memory_size_mb,registrations,returned
count,342332.000000,342332.000000,342332.000000,342332.000000,342332.000000,342332.000000,342332.000000,340442.000000,340442.000000,342332.000000,342332.000000
mean,1.577337,1.028610,1.538071,1.272867,2.927082,1.105301,2.326283,359.056964,3996.317517,1.567829,0.406874
std,0.804206,0.892683,0.566776,0.573181,1.028048,0.306942,2.551360,88.342857,1896.398925,1.651584,0.716612
min,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,80.000000,452.000000,1.000000,0.000000
25%,1.000000,0.000000,1.000000,1.000000,2.000000,1.000000,1.000000,315.000000,2789.000000,1.000000,0.000000
50%,1.000000,1.000000,2.000000,1.000000,3.000000,1.000000,2.000000,320.000000,3680.000000,1.000000,0.000000
75%,2.000000,2.000000,2.000000,2.000000,4.000000,1.000000,3.000000,440.000000,5572.000000,1.000000,1.000000
max,3.000000,2.000000,2.000000,3.000000,4.000000,2.000000,13.000000,999.000000,17205.000000,70.000000,22.000000


In [5]:
df.head()

,date,registration_type,played_t11_before,registration_channel,network_type,device_tier,device_type,device_manufacturer,screen_dpi,device_memory_size_mb,device_model,os_version,registrations,returned
0,2022-06-02,3,0,1,1,3,1,2,320.0,2820.0,Xiaomi Redmi 7,Android OS 10 / API-29,1,1
1,2022-06-02,1,2,1,1,4,1,0,480.0,2740.0,OMIX OMIX X300,Android OS 11 / API-30,1,0
2,2022-06-02,3,0,1,1,3,1,1,315.0,3571.0,samsung SM-M215F,Android OS 11 / API-30,1,0
3,2022-06-02,1,2,2,1,2,1,10,320.0,3774.0,TECNO MOBILE LIMITED TECNO KC3,Android OS 9 / API-28,1,1
4,2022-06-02,1,2,2,1,1,1,1,320.0,889.0,samsung SM-A013G,Android OS 10 / API-29,1,0


- We can't shuffle data because we have time series data and some date have more rows
- we have missing values and non numeric values. I think i'll firstly convert all data into numbers, split and then 

In [8]:
df.screen_dpi.median()

320.0

In [9]:
df.screen_dpi.mode()

0    320.0
Name: screen_dpi, dtype: float64

In [14]:
df.os_version.value_counts()

Android OS 11 / API-30       104801
Android OS 10 / API-29        80577
Android OS 12 / API-31        54064
Android OS 9 / API-28         35797
Android OS 8.1.0 / API-27     16038
                              ...  
Android OS 5.0 / API-19           1
Android OS 8.1.1 / API-22         1
Android OS 4.4 / API-19           1
Android OS 7.0.0 / API-22         1
Android OS 10.0 / API-28          1
Name: os_version, Length: 79, dtype: int64

320.0    70828
440.0    70594
480.0    43813
315.0    30396
280.0    20528
         ...  
548.0        1
146.0        1
532.0        1
358.0        1
520.0        1
Name: screen_dpi, Length: 355, dtype: int64

In [16]:
df.tail()

,date,registration_type,played_t11_before,registration_channel,network_type,device_tier,device_type,device_manufacturer,screen_dpi,device_memory_size_mb,device_model,os_version,registrations,returned
342327,2022-09-15,1,1,2,2,2,1,11,480.0,2848.0,LENOVO Lenovo K53a48,Android OS 7.0 / API-24,1,0
342328,2022-09-15,2,0,2,1,4,1,1,420.0,7443.0,samsung SM-N985F,Android OS 12 / API-31,3,0
342329,2022-09-15,1,1,1,1,3,1,4,320.0,3688.0,HUAWEI MAR-LX1A,Android OS 10 / API-29,1,0
342330,2022-09-15,1,1,1,1,4,1,0,480.0,3748.0,OMIX OMIX X500,Android OS 11 / API-30,2,0
342331,2022-09-15,1,1,2,2,4,1,2,440.0,5613.0,Xiaomi M2101K7BG,Android OS 11 / API-30,2,0


In [18]:
df.device_model.value_counts()

Xiaomi Redmi Note 8 Pro    7380
Xiaomi Redmi Note 8        6643
samsung SM-G610F           5001
Xiaomi M2003J15SC          4735
Xiaomi Redmi Note 9 Pro    4050
                           ... 
samsung SM-J410G              1
LEFE Blade X9                 1
OnePlus IV2201                1
SNCOIA QT19101                1
HUAWEI VKY-AL00               1
Name: device_model, Length: 3236, dtype: int64

In [20]:
df.device_memory_size_mb.mode()

0    3679.0
Name: device_memory_size_mb, dtype: float64

In [21]:
df.device_memory_size_mb.median()

3680.0

 - So missing values in columns 'device_memory_size' and 'screen_dpi' we will fill up with median value
 - And rows with missing device_model and os_version I am going to delete, because I am not sure how it will affect on result if I fill it up with the most common values
 - we have a enough data, so I know that for machine learning is better to have more data but I will try on this way

In [26]:
df['device_memory_size_mb'] = df['device_memory_size_mb'].fillna(df['device_memory_size_mb'].median())


In [31]:
df['screen_dpi'] = df['screen_dpi'].fillna(df['screen_dpi'].median())


In [32]:
df.isna().sum()

date                        0
registration_type           0
played_t11_before           0
registration_channel        0
network_type                0
device_tier                 0
device_type                 0
device_manufacturer         0
screen_dpi                  0
device_memory_size_mb       0
device_model             1890
os_version               1890
registrations               0
returned                    0
dtype: int64

In [34]:
df.dropna(axis=0, how='any', inplace=True)

In [35]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 340442 entries, 0 to 342331
Data columns (total 14 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   date                   340442 non-null  object 
 1   registration_type      340442 non-null  int64  
 2   played_t11_before      340442 non-null  int64  
 3   registration_channel   340442 non-null  int64  
 4   network_type           340442 non-null  int64  
 5   device_tier            340442 non-null  int64  
 6   device_type            340442 non-null  int64  
 7   device_manufacturer    340442 non-null  int64  
 8   screen_dpi             340442 non-null  float64
 9   device_memory_size_mb  340442 non-null  float64
 10  device_model           340442 non-null  object 
 11  os_version             340442 non-null  object 
 12  registrations          340442 non-null  int64  
 13  returned               340442 non-null  int64  
dtypes: float64(2), int64(9), object(3)
m

In [40]:
df.isnull().sum(), df.shape

(date                     0
 registration_type        0
 played_t11_before        0
 registration_channel     0
 network_type             0
 device_tier              0
 device_type              0
 device_manufacturer      0
 screen_dpi               0
 device_memory_size_mb    0
 device_model             0
 os_version               0
 registrations            0
 returned                 0
 dtype: int64,
 (340442, 14))

In [76]:
df['month'] = pd.DatetimeIndex(df['date']).month
df['day_of_week'] = pd.DatetimeIndex(df['date']).dayofweek

In [77]:
df.isnull().sum(), df.shape

(date                     0
 registration_type        0
 played_t11_before        0
 registration_channel     0
 network_type             0
 device_tier              0
 device_type              0
 device_manufacturer      0
 screen_dpi               0
 device_memory_size_mb    0
 device_model             0
 os_version               0
 registrations            0
 returned                 0
 month                    0
 day_of_week              0
 dtype: int64,
 (340442, 16))

In [64]:
# So now we can split data into train and test set

In [79]:
df.tail()

,date,registration_type,played_t11_before,registration_channel,network_type,device_tier,device_type,device_manufacturer,screen_dpi,device_memory_size_mb,device_model,os_version,registrations,returned,month,day_of_week
342327,2022-09-15,1,1,2,2,2,1,11,480.0,2848.0,LENOVO Lenovo K53a48,Android OS 7.0 / API-24,1,0,9,3
342328,2022-09-15,2,0,2,1,4,1,1,420.0,7443.0,samsung SM-N985F,Android OS 12 / API-31,3,0,9,3
342329,2022-09-15,1,1,1,1,3,1,4,320.0,3688.0,HUAWEI MAR-LX1A,Android OS 10 / API-29,1,0,9,3
342330,2022-09-15,1,1,1,1,4,1,0,480.0,3748.0,OMIX OMIX X500,Android OS 11 / API-30,2,0,9,3
342331,2022-09-15,1,1,2,2,4,1,2,440.0,5613.0,Xiaomi M2101K7BG,Android OS 11 / API-30,2,0,9,3
